In [1]:
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
from sklearn.decomposition import PCA

### reading directory

In [2]:
data = pd.DataFrame()
data['image-name'] = os.listdir('../data/flowers/')
data['species'] = data['image-name'].apply(lambda x: int(x[:2]))

### opening images

In [3]:
raw_images_flatten = np.array([plt.imread('../data/flowers/'+img).ravel() for img in data['image-name'].tolist()])

### applying pca

In [4]:
def reduce_dim(imgs, k):
    pca = PCA(n_components=k)
    return pca.fit_transform(imgs)

images_600 = reduce_dim(raw_images_flatten, 600)

### using a neural network to create a better mapping

In [5]:
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Flatten, Dense, concatenate,  Dropout, Conv2D, MaxPool2D

input_image = Input(shape=(128, 128, 4))
x = Conv2D(32, (3, 3), activation='relu')(input_image)
x = MaxPool2D((2, 2))(x)
x = Conv2D(16, (3, 3), activation='relu')(x)
x = MaxPool2D((2, 2))(x)
x = Conv2D(8, (3, 3), activation='relu')(x)
x = MaxPool2D((2, 2))(x)
x = Flatten()(x)
x = Dense(64, activation='relu')(x)

base_network = Model(inputs=input_image, outputs=x)


input_image_one = Input(shape=(128, 128, 4), name='input_image_one') # input layer for image one
input_image_two = Input(shape=(128, 128, 4), name='input_image_two') # input layer for image two
concat_layer = concatenate([base_network(input_image_one), base_network(input_image_two)])
x = Dense(32)(concat_layer)
similarity = Dense(1, activation='sigmoid')(x) # similar or not

model = Model(inputs=[input_image_one, input_image_two],
            outputs=similarity)
model.compile(loss='binary_crossentropy', optimizer='rmsprop')

In [6]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_image_one (InputLayer)    [(None, 128, 128, 4) 0                                            
__________________________________________________________________________________________________
input_image_two (InputLayer)    [(None, 128, 128, 4) 0                                            
__________________________________________________________________________________________________
model (Functional)              (None, 64)           107384      input_image_one[0][0]            
                                                                 input_image_two[0][0]            
__________________________________________________________________________________________________
concatenate (Concatenate)       (None, 128)          0           model[0][0]                

In [7]:
data_reduced = data.sample(100)

In [8]:
first_input = []
second_input = []
labels = []
for i, img_one in enumerate(data_reduced['image-name'].to_list()):
    for j, img_two in enumerate(data_reduced['image-name'].to_list()):
        
        if j > i:
            first_input.append(plt.imread('../data/flowers/'+img_one))
            second_input.append(plt.imread('../data/flowers/'+img_two))
            s = 0 if img_one[:2] != img_two[:2] else 1
            labels.append(s)

first_input = np.array(first_input)
second_input = np.array(second_input)  
labels = np.array(labels)  

In [9]:
model.fit([first_input, second_input], labels, epochs=5)

Epoch 1/5
155/155 [==============================] - 36s 231ms/step - loss: 0.2934
Epoch 2/5
155/155 [==============================] - 35s 228ms/step - loss: 0.2307
Epoch 3/5
155/155 [==============================] - 35s 228ms/step - loss: 0.2333
Epoch 4/5
155/155 [==============================] - 35s 227ms/step - loss: 0.2164
Epoch 5/5
155/155 [==============================] - 35s 228ms/step - loss: 0.2365


In [10]:
raw_images = np.array([plt.imread('../data/flowers/'+img) for img in data['image-name'].tolist()])

In [11]:
base_network.predict(raw_images)[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 1.1384286 ,
       0.        , 0.        , 0.        , 0.        , 0.19039662,
       0.        , 0.        , 0.07191174, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.04242852, 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.21581745, 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.8016604 , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.7161391 , 0.9376043 , 0.        , 0.7023733 ,
       0.        , 0.7692271 , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        ], dtype=float32)